# Project#2



Using only NumPy and Pandas create a Neural Network from scratch(90 pt)
<br> Set network architecture as follows:
* Implement input, hidden, and output layers concerning input-output shape.
* Define activation function.
* Implement FeedForward
* Implement BackPropagation
* Implement Train and Test functions.




Test your model on both datasets then calculate the confusion matrix and accuracy
* MNIST Dataset


importing only pandas and numby

In [ ]:

import numpy as np
import pandas as pd


Define error function

In [ ]:

def loss(y_true, y_pred):
    # Calculate the mean squared error loss between y_true and y_pred.
    mse = np.mean(np.square(y_true - y_pred))
    return mse

def lossPrime(y_true, y_pred):
    # Calculate the mean squared error loss between y_true and y_pred.
    mse_prime =2*(y_pred - y_true) // np.size(y_true);
    return mse_prime



In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

def softmax_derivative( x):
    return x * (1 - x)

Neural Network class

In [ ]:
#class neural network

class NeuralNetwork:

    def __init__(self, sizeOfInput, sizeOfhidden, sizeOfOutput, activationFunction='sigmoid'):
        self.sizeOfInput = sizeOfInput
        self.sizeOfhidden = sizeOfhidden
        self.sizeOfOutput = sizeOfOutput
        self.activationFunction = activationFunction
        #here I initialize my weights randomly
        self.wOfInputs, self.bOfHidden, self.wOfOutput, self.bOfOutput = \
            self.InitializeWeightsRandomly()

#Define some activation function and thier derivative

    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self,x):
        return x * (1 - x)

    def tanh(self,x):
        return np.tanh(x)

    def tanh_derivative(self,x):
        return 1 - np.tanh(x) ** 2

    def relu(self,x):
        return np.maximum(0, x)

    def relu_derivative(self,x):
        return np.where(x > 0, 1, 0)

    def softmax(self, x):
        exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def softmax_derivative(self, x):
        return x * (1 - x)


    #this is the function that initialize them randomly
    def InitializeWeightsRandomly(self):
        wOfInputs = np.random.randn(self.sizeOfInput, self.sizeOfhidden)
        bOfHidden = np.zeros((1, self.sizeOfhidden))
        wOfOutput = np.random.randn(self.sizeOfhidden, self.sizeOfOutput)
        bOfOutput = np.zeros((1, self.sizeOfOutput))
        return wOfInputs, bOfHidden, wOfOutput, bOfOutput

    #the forward path
    def forward(self, inputs):
      HiddenInputs = np.dot(inputs, self.wOfInputs) + self.bOfHidden
      if self.activationFunction == 'sigmoid':
          HiddenOutput = sigmoid(HiddenInputs)
      elif self.activationFunction == 'tanh':
          HiddenOutput = tanh(HiddenInputs)
      elif self.activationFunction == 'relu':
          HiddenOutput = relu(HiddenInputs)
      else:
          raise ValueError("Invalid activation function")

      OutputOfInputs = np.dot(HiddenOutput, self.wOfOutput) + self.bOfOutput
      FinalOutput = self.softmax(OutputOfInputs)

      return HiddenOutput, FinalOutput


    def backward(self, inputs, targets, HiddenOutput, FinalOutput): #backward

        Error = targets - FinalOutput #first calculate error
        DerivativeError = Error * self.softmax_derivative(FinalOutput) #then the derivative error with respect to oj

        #then calculating the error in the hidden layer
        if self.activationFunction == 'sigmoid':
            ErrorInHidden = DerivativeError.dot(self.wOfOutput.T)
            DerivativeErrorInHidden = ErrorInHidden * self.sigmoid_derivative(HiddenOutput)
        elif self.activationFunction == 'tanh':
            ErrorInHidden = DerivativeError.dot(self.wOfOutput.T)
            DerivativeErrorInHidden = ErrorInHidden * self.tanh_derivative(HiddenOutput)
        elif self.activationFunction == 'relu':
            ErrorInHidden = DerivativeError.dot(self.wOfOutput.T)
            DerivativeErrorInHidden = ErrorInHidden * self.relu_derivative(HiddenOutput)
        else:
            raise ValueError("Invalid activation function")

        return DerivativeErrorInHidden, DerivativeError

   #after calcultaing the error we need to get new w and b and update it
    def UpdateNewWeights(self, inputs, DerivativeErrorInHidden, DerivativeError, HiddenOutput, LearningRate):
        self.wOfOutput += HiddenOutput.T.dot(DerivativeError) * LearningRate
        self.bOfOutput += np.sum(DerivativeError, axis=0, keepdims=True) * LearningRate
        self.wOfInputs += inputs.T.dot(DerivativeErrorInHidden) * LearningRate
        self.bOfHidden += np.sum(DerivativeErrorInHidden, axis=0, keepdims=True) * LearningRate


    def train(self, inputs, targets, epochs=1000, LearningRate=0.01):
        for epoch in range(epochs):
            for i in range(len(inputs)):
                input_data = np.array(inputs[i]).reshape(1, -1)
                target_data = np.array(targets[i]).reshape(1, -1)

                HiddenOutput, FinalOutput = self.forward(input_data)

                DerivativeErrorInHidden, DerivativeError = self.backward(input_data, target_data, HiddenOutput, FinalOutput)

                self.UpdateNewWeights(input_data, DerivativeErrorInHidden, DerivativeError, HiddenOutput, LearningRate)

    def test(self, inputs):
        predictions = []
        for i in range(len(inputs)):
            input_data = np.array(inputs[i]).reshape(1, -1)
            _, FinalOutput = self.forward(input_data)
            predictions.append(FinalOutput[0])

        return predictions

    def evaluate(self, inputs, targets):
        predictions = self.test(inputs)
        correct_predictions = 0

        for i in range(len(predictions)):
            predicted_class = np.argmax(predictions[i])
            true_class = np.argmax(targets[i])

            if predicted_class == true_class:
                correct_predictions += 1

        accuracy = correct_predictions / len(targets)
        return accuracy


Test On MNIST

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


mnist_inputs_train = train_images.reshape((60000, 28 * 28)).astype('float32') / 255.0
mnist_inputs_test = test_images.reshape((10000, 28 * 28)).astype('float32') / 255.0


mnist_targets_train = LabelBinarizer().fit_transform(train_labels)
mnist_targets_test = LabelBinarizer().fit_transform(test_labels)


mnist_inputs_train, mnist_inputs_test, mnist_targets_train, mnist_targets_test = train_test_split(
    mnist_inputs_train, mnist_targets_train, test_size=0.2, random_state=42)

mnist_nn = NeuralNetwork(sizeOfInput=784, sizeOfhidden=128, sizeOfOutput=10, activationFunction='sigmoid')
mnist_nn.train(mnist_inputs_train, mnist_targets_train, epochs=50, LearningRate=0.1)


mnist_accuracy = mnist_nn.evaluate(mnist_inputs_test, mnist_targets_test)
print("MNIST Accuracy:", mnist_accuracy * 100, "%")


Confusion matrix of MNIST

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


mnist_predictions = mnist_nn.test(mnist_inputs_test)
mnist_predicted_labels = np.argmax(mnist_predictions, axis=1)
mnist_true_labels = np.argmax(mnist_targets_test, axis=1)


mnist_conf_matrix = confusion_matrix(mnist_true_labels, mnist_predicted_labels)

print("Confusion Matrix for MNIST:")
print(mnist_conf_matrix)


Confusion Matrix for MNIST:
[[1067    0   11   12    4   22   27    6   20    6]
 [   0 1259   16    9    3    2    5    7   17    4]
 [  17   21  942   33   25   10   43   20   51   12]
 [  17    6   47  944    5   59   17   18   74   32]
 [   8    1   10    4  936   10   30   10   32  135]
 [  36    9   21   89   26  781   23   15   86   18]
 [  19    8   21    2   14   17 1063    2   27    4]
 [  11   22   28   21   14    9    1 1110   19   64]
 [  12   10   43   83   18   64   17   10  881   22]
 [   9    2   11   23   90   15    6   65   31  942]]


### ***Using built in libraries***

---




Keras Neural Network with MNIIST

In [ ]:

NeuralNetworkKeras_MNIST = Sequential()
NeuralNetworkKeras_MNIST.add(Dense(units=128, activation='relu', input_dim=784))
NeuralNetworkKeras_MNIST.add(Dense(units=10, activation='softmax'))
NeuralNetworkKeras_MNIST.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

mnist_targets_train_categorical = mnist_targets_train
mnist_targets_test_categorical = mnist_targets_test

NeuralNetworkKeras_MNIST.fit(mnist_inputs_train, mnist_targets_train_categorical, epochs=5, batch_size=32, verbose=0)
mnist_accuracy_keras = NeuralNetworkKeras_MNIST.evaluate(mnist_inputs_test, mnist_targets_test_categorical, verbose=0)[1]


print("MNIST Accuracy in my NeuralNetwork:",mnist_accuracy * 100, "%")
print("MNIST Accuracy in Keras Neural Network:",mnist_accuracy_keras * 100, "%")

MNIST Accuracy in my NeuralNetwork: 82.70833333333333 %
MNIST Accuracy in Keras Neural Network: 94.9583351612091 %


Sklearn neural network with MNIST

In [ ]:
from keras.utils import to_categorical

NeuralNetworkSklearn_MNIST = MLPClassifier(hidden_layer_sizes=(128,), max_iter=5, random_state=42)
NeuralNetworkSklearn_MNIST.fit(mnist_inputs_train, mnist_targets_train)
PredictMnist= NeuralNetworkSklearn_MNIST.predict(mnist_inputs_test)
mnist_accuracy_sklearn = accuracy_score(mnist_targets_test, PredictMnist)
mnist_targets_test_reshaped = np.array(to_categorical(mnist_targets_test)).reshape(len(mnist_targets_test), -1)
print("MNIST Accuracy in my NeuralNetwork:",mnist_accuracy * 100, "%")
print("MNIST Accuracy in MLPClassifier:", mnist_accuracy_sklearn * 100, "%")

MNIST Accuracy in my NeuralNetwork: 82.70833333333333 %
MNIST Accuracy in MLPClassifier: 92.33333333333333 %


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
